# Planning-Lab Lesson 1: Search Strategies

In this first lesson we will work both on Search Strategies. 

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

In order to use the environment we need first to import the packages of OpenAI Gym. Notice that due to the structure of this repository, we need to add the parent directory to the path

In [1]:
import os, sys, time, math

module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *
import gym, envs

# Uninformed search
### Assignment 1: Breadth-First Search (BFS)

**Your first assignment is to implement the BFS algorithm on SmallMaze. In particular, given the *tree search* version of BFS, you should implement the *graph search* version.**

The results returned by your **BFS** must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

**For the time_cost, we consider a node checked after its generation!**

After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- BFS_GraphSearch(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for *Graph Search* and *Tree Search*:

<img src="images/tree-graph-search.png" width="600">

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *BFS* algorithm, note that it refers to the implementation of the *Graph Search Version*:

<img src="images/BFS2.png" width="600">

**Here's an implementation of the BFS algorithm in its Tree Search version:**

In [2]:
def BFS_TreeSearch(problem):
    """
    Tree Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1
    
    if node.state == problem.goalstate:
        return build_path(node), time_cost, space_cost
    
    frontier = NodeQueue()
    frontier.add(node)    
    
    while not frontier.is_empty():
        node = frontier.remove()
        
        for action in range(problem.action_space.n):
            
            child = Node(problem.sample(node.state, action), node)
            time_cost += 1
            
            if problem.goalstate == child.state: 
                return build_path(child), time_cost, space_cost
            
            frontier.add(child)
             
        space_cost = max(space_cost, len(frontier))  

    return None, time_cost, space_cost

**Now you can implement the same BFS algorithm in the Graph Search version:**

In [3]:
def BFS_GraphSearch(problem):
    """
    Graph Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1
    
    if node.state == problem.goalstate:
        return build_path(node), time_cost, space_cost
    
    frontier = NodeQueue()
    explored = set()
    frontier.add(node)
    
    while not frontier.is_empty():
        node = frontier.remove()
        explored.add(node.state)
        
        for action in range(problem.action_space.n):
            child = Node(problem.sample(node.state, action), node)
            time_cost += 1
            
            if not (child.state in explored or child.state in frontier):
                if problem.goalstate == child.state: 
                    return build_path(child), time_cost, space_cost
                frontier.add(child)
             
        space_cost = max(space_cost, len(frontier) + len(explored))  

    return None, time_cost, space_cost


**The following code calls your graph search version and the tree search version of BFS and checks the results:**

In [4]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = BFS_TreeSearch(environment)
solution_gs, time_gs, memory_gs = BFS_GraphSearch(environment)

results = CheckResult_L1A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  BFS TREE SEARCH PROBLEM  ########
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 103723
Max n° of nodes in memory: 77791

##########################################
#######  BFS Graph SEARCH PROBLEM  #######
##########################################
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 59
Max n° of nodes in memory: 15

===> Your solution is correct!



## Assignment 2:  Depth-Limited Search (DLS) and Iterative Deepening depth-first Search (IDS)

**Your second assignment is to implement the IDS algorithm on SmallMaze. 
In particular, given Deph Limited Search (*DLS*) in the *tree search* version, you are required to implement the Ierative Depening Search (*IDS*) function.**

Similarly to assignment 1, the results returned by your ids must be in the following form (path, Time Cost, Space Cost) described above. After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- IDS(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *Depth-Limited Search (Tree Search Version)* and *Iterative deepening depth-first search (Tree Search Version)*:

<img src="images/dls.png" width="600">

<img src="images/ids.png" width="600">

Note that Node() has a depthcost variable that represents the depth of the node in the search tree. This variable is automatically set by the Node constructor: if the root node has a depthcost of 0, its children will have a depthcost increased by 1. The depthcost is useful to compute the space cost for IDS. An example is shown below:

In [5]:
start = environment.startstate
root = Node(start)  # parent = None and depthcost = 0 as default
child = Node(environment.sample(start, 0), root) # the depthcost is set automatically in the Node constructor
print("Root depthcost: {}\tChild depthcost: {}".format(root.depthcost, child.depthcost))

Root depthcost: 0	Child depthcost: 1


In [6]:
def DLS(problem, limit, RDLS_Function):
    """
    DLS
    
    Args:
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    node = Node(problem.startstate, None)
    return RDLS_Function(node, problem, limit, set())

In [7]:
def Recursive_DLS_TreeSearch(node, problem, limit, explored=None):
    """
    DLS (Tree Search Version)
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    if problem.goalstate == node.state: # Goal state check
        return build_path(node), 1, node.depthcost
    elif limit == 0: # Limit budget check for cutoff
        return "cut_off", 1, node.depthcost
    
    space_cost = node.depthcost
    time_cost = 1 
    
    cut_off_occurred = False
    for action in range(problem.action_space.n): # Look around
        child = Node(problem.sample(node.state, action), node) # Child node 
        result = Recursive_DLS_TreeSearch(child, problem, limit-1) # Recursive call
        time_cost += result[1]

        space_cost = max(space_cost, result[2])
        
        if result[0] == "cut_off":
            cut_off_occurred = True
        elif result[0] != "failure": # Solution found
            return result[0], time_cost, space_cost
        
    if(cut_off_occurred): # Solution not found but cutoff occurred
        return "cut_off", time_cost, space_cost
    else: # No solution and no cutoff: failure
        return "failure", time_cost, space_cost

**Study the provided code for Depth Limited Search (DLS) and Recursive_DLS_TreeSearch and implement the Iterative Deepening Search function reported below.**

In [8]:
def IDS(problem, DLS_Function):
    """
    Iteartive_DLS DLS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    node = Node(problem.startstate, None)
    start = Node(problem.startstate)
    total_time_cost = 0
    total_space_cost = 1

    for depth in zero_to_infinity():
        result = DLS(problem, depth, DLS_Function)
        cut_off_occured = result[0] == "cut_off"
        
        total_time_cost += result[1]
        total_space_cost = max(total_space_cost, result[2])
        
        if (not cut_off_occured):
            return result[0], total_time_cost, total_space_cost, depth

    return None, total_time_cost, total_space_cost, 0

The following code calls your version of IDS and checks the results:

In [9]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts, iterations_ts = IDS(environment, Recursive_DLS_TreeSearch)

results = CheckResult_L1A2([solution_ts, time_ts, memory_ts, iterations_ts], environment)
results.check_sol_ts()

##########################################
#######  IDS TREE SEARCH PROBLEM  ########
##########################################
Necessary Iterations: 9
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 138298
Max n° of nodes in memory: 9

===> Your solution is correct!



#### Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?

# Quiz Uninformed Search
## Question 1
Consider the following environment called GrdMaze-v0:
```python
[
  ['C', 'C', 'C', 'S'],
  ['C', 'C', 'W', 'C'],
  ['C', 'C', 'C', 'C'],
  ['C', 'W', 'W', 'W'],
  ['C', 'C', 'C', 'G'],
]
startstate = 3
goalstate = 19

# You can create this environment with the following commands
envname = "GrdMaze-v0"
environment = gym.make(envname)
```
### Using BFS Graph Search, which one of the following answers is correct?
- [ ] a. Graph Search explores already visited states
- [ ] b. Graph Search explores 60 nodes before returning the solution
- [ ] c. Graph Search is not able to find the solution
- [x] d. Graph Search explores 59 nodes before returning the solution

In [10]:
def print_solution(solution, env):
    print("Your solution: {}".format(solution_2_string(solution[0], env)))
    print("N° of nodes explored: {}".format(solution[1]))
    print("Max n° of nodes in memory: {}".format(solution[2]))

In [40]:
envname = "GrdMaze-v0"
environment = gym.make(envname)
solution = BFS_GraphSearch(environment)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUESTION 1                               #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

print_solution(solution, environment)

########################################################
####### QUESTION 1                               #######
########################################################
Your solution: [(0, 2), (0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 59
Max n° of nodes in memory: 15


## Question 2
### Consider the GrdMaze-v0 environment using IDS with Recursive_DLS_TreeSearch, which one of the following answers is correct?
- [ ] a. IDS explores fewer nodes than in the SmallMaze-v0 environment
- [x] b. IDS returns a solution in 10 iterations
- [ ] c. IDS returns a solution in 12 iterations

In [41]:
solution = IDS(environment, Recursive_DLS_TreeSearch)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUESTION 2                               #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

print_solution(solution, environment)
print("Iterations: {}".format(solution[3]))

########################################################
####### QUESTION 2                               #######
########################################################
Your solution: [(0, 2), (0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 487824
Max n° of nodes in memory: 10
Iterations: 10


## Question 3
### Consider the GrdMaze-v0 env, what happens if Depth-Limited Search (DLS) is run with a depth limit of 7 for the goal depth?
- [x] a. The time cost is > 7200 with a "cut_off" result
- [ ] b. The time cost is < 7200 with a "cut_off" result
- [ ] c. The goal is reached with an optimal solution

In [45]:
solution = DLS(environment, 7, Recursive_DLS_TreeSearch)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUIZ QUESTION 3                          #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

print_solution(solution, environment)

########################################################
####### QUIZ QUESTION 3                          #######
########################################################
Your solution: cut_off
N° of nodes explored: 21845
Max n° of nodes in memory: 7


## Question 4
### Consider the GrdMaze-v0 env, what is the final space cost in order to obtain an optimal solution with Depth-Limited Search (DLS)?
- [ ] a. 5
- [ ] b. 11
- [x] c. 10
- [ ] d. 9

In [46]:
print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUIZ QUESTION 4                          #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)
expected_solution = [(0, 2), (0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)];

for limit in range(1,100):
    print("Checking limit: {}".format(limit))
    solution = DLS(environment, limit, Recursive_DLS_TreeSearch)

    if solution_2_string(solution[0], environment) == [(0, 2), (0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]:
        print_solution(solution, environment)
        print(bcolors.BOLD + bcolors.OKGREEN + 'Solution is optimal with limit: {}'.format(limit) + bcolors.ENDC)
        break

########################################################
####### QUIZ QUESTION 4                          #######
########################################################
Checking limit: 1
Checking limit: 2
Checking limit: 3
Checking limit: 4
Checking limit: 5
Checking limit: 6
Checking limit: 7
Checking limit: 8
Checking limit: 9
Checking limit: 10
Your solution: [(0, 2), (0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 21794
Max n° of nodes in memory: 10
Solution is optimal with limit: 10


## Question 5
Consider the BlockedMaze-v0 environment reported below.
```python
[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'C' 'W' 'W']
 ['C' 'C' 'W' 'G']]
startstate:  2
goalstate:  19
# You can create this environment with the commands:
envname = "BlockedMaze-v0"
environment = gym.make(envname)
```
### Using BFS Graph Search, which one of the following answers is correct?
- [x] a. The search process terminates correctly, indicating that there is no solution, and it explores 61 nodes
- [ ] b. The search process terminates correctly, indicating that a solution exists, and it explores 60 nodes
- [ ] c. The search process does not terminate

In [47]:
envname = "BlockedMaze-v0"
environment = gym.make(envname)
solution = BFS_GraphSearch(environment)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUESTION 5                               #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

print_solution(solution, environment)

########################################################
####### QUESTION 5                               #######
########################################################
Your solution: None
N° of nodes explored: 61
Max n° of nodes in memory: 15


## Question 6
### Consider the BlockedMaze-v0 env using Recursive_DLS_TreeSearch and setting the limit value for the exploration to 5. Which of the following answers is correct?
- [ ] a. The search returns a "cut_off" answer, exploring a total of 300 nodes
- [ ] b. The search returns a "cut_off" answer, and the method explores 460 nodes, i.e., more nodes w.r.t GrdMaze-v0 environment
- [x] c. The search returns a "cut_off" answer, and the method explores 453 nodes, i.e., the same number of nodes as in GrdMaze-v0 environment

In [55]:
solution_blocked = DLS(environment, 5, Recursive_DLS_TreeSearch)

print(bcolors.OKCYAN + '########################################################' + bcolors.ENDC)
print(bcolors.OKCYAN + '####### QUIZ QUESTION 6                          #######' + bcolors.ENDC)
print(bcolors.OKCYAN + '########################################################'+ bcolors.ENDC)

envname = "GrdMaze-v0"
environment = gym.make(envname)
solution_grd = DLS(environment, 5, Recursive_DLS_TreeSearch)

if solution_blocked > solution_grd:
    print("Blocked explores more nodes than Grid maze: {} > {}".format(solution_blocked[1], solution_grd[1]))
elif solution_blocked < solution_grd:
    print("Blocked explores less nodes than Grid maze: {} < {}".format(solution_blocked[1], solution_grd[1]))
elif solution_blocked == solution_grd:
    print("Blocked explores the same nodes as Grid maze: {} = {}".format(solution_blocked[1], solution_grd[1]))


########################################################
####### QUIZ QUESTION 6                          #######
########################################################
Blocked explores the same nodes as Grid maze: 1365 = 1365
